In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import os
import pandas as pd
from sqlalchemy import create_engine
from dotenv import load_dotenv

# Load credentials from .env file
load_dotenv()

DB_USER = os.getenv("DB_USER")
DB_PASSWORD = os.getenv("DB_PASSWORD")
DB_HOST = os.getenv("DB_HOST")
DB_PORT = os.getenv("DB_PORT")
DB_NAME = os.getenv("DB_NAME")

# Create database connection URL (for MySQL; change driver if needed)
db_url = f"mysql+pymysql://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}"

# Create SQLAlchemy engine
engine = create_engine(db_url)

In [5]:
# Step 1: Load base tables
financialrequests = pd.read_sql("SELECT Id, CreatorId, ApprovedAmount FROM financialrequests", engine)
abpusers = pd.read_sql("SELECT Id, UserName, ManagerId FROM abpusers", engine)

In [6]:
# Step 2: Join financialrequests with abpusers on CreatorId = u.Id
df = financialrequests.merge(
    abpusers,
    left_on="CreatorId",
    right_on="Id",
    how="inner"
)
df

,Id_x,CreatorId,ApprovedAmount,Id_y,UserName,ManagerId
0,3a08d703-fa37-56d4-cdd2-794b2ab38241,3a074590-6ced-5665-38bd-a50c9105bf36,1000.0,3a074590-6ced-5665-38bd-a50c9105bf36,sobhy,None
1,3a08d726-ba65-0faa-720e-839c2bbd779b,3a074590-6ced-5665-38bd-a50c9105bf36,326.0,3a074590-6ced-5665-38bd-a50c9105bf36,sobhy,None


In [23]:
# Step 3: Aggregate (GROUP BY u.Id, u.Name)
agg_df = df.groupby(
    ["Id_y"], as_index=False
).agg(
    TotalIssuedAmount=("ApprovedAmount", "sum")
).rename(
    columns={
        "Id_y": "OfficerId"
    }
).merge(
    abpusers,
    left_on="OfficerId",
    right_on="Id",
    how="left"
).drop(
    columns=["Id"]
)
agg_df

,OfficerId,TotalIssuedAmount,UserName,ManagerId
0,3a074590-6ced-5665-38bd-a50c9105bf36,1326.0,sobhy,None


In [24]:
# Step 4: Rename / format columns
final_df = agg_df.rename(columns={
    "UserName": "OfficerName",
})

# Step 5: Order by TotalIssuedAmount DESC
final_df = final_df.sort_values("TotalIssuedAmount", ascending=False).reset_index(drop=True)
final_df

,OfficerId,TotalIssuedAmount,OfficerName,ManagerId
0,3a074590-6ced-5665-38bd-a50c9105bf36,1326.0,sobhy,None
